# Python bindings

C++ and Python are are strong combination: With C++ we can program for maximal performance, with Python we can work with objects in a very convenient way. Both share a similar object oriented paradigm.

Python bindings allow to use C++ functions and classes from Python.

A very popular library for wrapping C++ objects to Python is
[pybind11](https://pybind11.readthedocs.io/en/stable/).


Now you have to install [Python](https://www.python.org). Choose a recent version (at least Python 3.8 should be ok). Using conda is also fine, you have to replace `pip install` by `conda-install` in the following.

Install pybind11 as a Python package:

    pip install pybind11


Clone the pybind-branch from the ASC-git:

    git clone --branch pybind https://github.com/TUWien-ASC/ASC-bla.git


For building and installing our Python package we use [scikit-build](https://scikit-build.readthedocs.io/en/latest/) which can be easily installed using 

    pip install scikit-build

Building, installing and testing ASC-bla should now work with

    cd ASC-git
    pip install . -v
    cd py_tests
    python py_tests

## Binding C++ classes to Python:

All code for wrapping the classes and functions happens in *src/bind_bla.cpp*:

```cpp
#include <pybind11/pybind11.h>
#include "vector.h"

using namespace ASC_bla;
namespace py = pybind11;

PYBIND11_MODULE(bla, m) {
    m.doc() = "Basic linear algebra module"; // optional module docstring
    
    py::class_<Vector<double>> (m, "Vector")
      .def(py::init<size_t>(), 
           py::arg("size"), "create vector of given size)
      .def("__len__", &Vector<double>::Size, 
           "return size of vector")
      ...
}
```

* we include the pybind11 headers, and abbreviate the pybind11 namespace as py
* PYBIND11_MODULE is a macro setting up the module *bla*, we can add members to it using the variable m.
* `py::class_<Vector<double>> (m, "Vector")` wraps the C++ class `Vector<double>` to Python, where its name is `Vector`.
* With `def` we can implement member functions and operators. We give the name of the function, the C++ function (which may be a old-style function pointer, or a lambda-function), name the arguments, and provide the documentation
* `py::init<size_t>()` is a special syntax for the constructor, in this case for the ctor with one size_t argument.
* the function `__len__` is called from the Python `len(v)` built in function

### setter/getter functions:

```cpp
    .def("__setitem__", [](Vector<double> & self, size_t i, double v) { self(i) = v; })
    .def("__getitem__", [](Vector<double> & self, size_t i) { return self(i); })

    .def("__setitem__", [](Vector<double> & self, py::slice inds, double val)
      {
        size_t start, stop, step, n;
        if (!inds.compute(self.Size(), &start, &stop, &step, &n))
          throw py::error_already_set();
        self.Range(start, stop).Slice(0,step) = val;
      })
```

The bracket operators `v[i] = val` or `print (v[j])` call the `__setitem__` and `__getitem__` methods with an `size_t` argument. The Python slice operator `v[3:7] = 0` calls the `__setitem__` method with an `py::slice` argument. 

## Importing the python module
We can now import the python module bla from the package ASCsoft. Either in the plane *.py* Python file, or into jupyter notebooks:

In [ ]:
from ASCsoft.bla import *

In [ ]:
x = Vector(5)
y = Vector(5)

for i in range(len(x)):
    x[i] = i
y[:] = 3

In [ ]:
print ("x+y =",x+y)

## Exercise

* Wrap your `Matrix<double,RowMajor>` to Python. Add getter/setter functions and operators.
* Pickling is the standard Python serialization (file io, parallel communication). Add pickling support to your classes. Use `py::bytes` to store the data.
* Numpy is Python standard for data exchange in scientific computing. Try to convert your Python `Vector`/`Matrix` to a numpy array using `np.asarray(v)`. How does it work ?  How efficient ? Add a [Buffer protocol](https://pybind11.readthedocs.io/en/stable/advanced/pycpp/numpy.html?highlight=buffer#buffer-protocol).

## Building the Python-package

You found two new files *setup.py* and *pyproject.toml*. They are responsible for building and installing a Python package. If we call 

    pip install .

the setup function from the file *setup.py* get called. It first triggers cmake, which installs everything in the directory *ASCsoft*. cmake knows nothing about the anatomy of a Python package. Here, [scikit-build](https://scikit-build.readthedocs.io/en/latest/skbuild.html) steps in ...

When everything is uploaded properly to github, everyone can build and install our library as a Python package by running pip install with the github url:

    pip install git+https://github.com/TUWien-ASC/ASC-bla.git@pybind


some links: [pip-tutorial](https://github.com/MichaelKim0407/tutorial-pip-package)